# Indroduction to Sagemaker Experiment Logger for PyTorch-Lightning
 This introduction shows the usage of the [SagemakerExperimentsLogger](https://github.com/tsenst/lightning-experiments-logger/blob/main/experiments_addon/logger.py#L44) on the example of simple MNIST classification using [Trainer](https://lightning.ai/docs/pytorch/stable/common/trainer.html) class of the PyTorch Lightning API.

## Setup
To run the notebook to following requirements.txt has to be installed

In [ ]:
! pip install --quiet "seaborn" "pytorch-lightning>=1.4, <2.0.0" "torchvision" "setuptools==67.4.0" "lightning>=2.0.0rc0" "ipython[notebook]>=8.0.0, <8.12.0" "pandas" "torchmetrics >=0.11.0" "torch>=1.8.1, <1.14.0" "torchmetrics>=0.7, <0.12"

## Prerequisite
At first the dataset has to be loaded. The used model has been introduced by the [Simple MNIST Model](https://lightning.ai/docs/pytorch/stable/notebooks/lightning_examples/mnist-hello-world.html) example of the Pytorch-Lightning page.


In [1]:
import os

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from sagemaker.experiments.run import Run

from example.model import MNISTModel
from experiments_addon.logger import SagemakerExperimentsLogger

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
mnist_model = MNISTModel(data_dir=PATH_DATASETS)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-xubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/tobias.senst/.config/sagemaker/config.yaml


## How to use
The log with [SagemakerExperimentsLogger](https://github.com/tsenst/lightning-experiments-logger/blob/main/experiments_addon/logger.py#L44) to AWS SageMaker Experiments. Three different use cases will be discussed:
1. Use logger inside the run context
2. Explicitly passing *run_name* and *experiment_name*
3. Use logger inside [training job](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html)
4. Use logger with other loggers

Sagemaker Experiments are organized by experiments and run. A run is a subunit of an experiment. When tracking the metrics of a training run the run (given by *run_name*) and experiment (given by *experiment_name*  has to be provided. For the experiment tracking SageMake API defines a Run Context within the metric writing code has been implemented.
 
### Use the SagemakerExperimentsLogger in a notebook within a run context
For the use in notebook it is recommendation to [create a Run object](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments-create.html). All log methods have to be called inside *with* statement. Because the *Trainer* is calling the log methods of the logger, it needs to be within the *with* statement too.

In [ ]:
with Run(experiment_name="testExperiment", run_name="testRun1"):
    logger = SagemakerExperimentsLogger()
    trainer = Trainer(
        logger=logger,
        devices=1,
        max_epochs=3,
    )
    trainer.fit(mnist_model)
    trainer.test()

### Use SagemakerExperimentsLogger by explicitly passing in run_name and experiment_name
There may be cases where the *with Run* statement is not practical. For that reason it is possible to obtain the run context at each log operation. To run the *SagemakerExperimentsLogger* without the *with* statement provide *experiment_name* and *run_name* when creating the object. 

In [3]:
logger = SagemakerExperimentsLogger(
    experiment_name="TestExp",
    run_name="TestRun"
)

trainer = Trainer(
    logger=logger,
    devices=1,
    max_epochs=3,
)
trainer.fit(mnist_model)
trainer.test()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | train_metrics | MetricCollection | 0     
1 | val_metrics   | MetricCollection | 0     
2 | test_metrics  | MetricCollection | 0     
3 | l1            | Linear           | 7.9 K 
---------------------------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-xubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/tobias.senst/.config/sagemaker/config.yaml


NoCredentialsError: Unable to locate credentials

### Use the SagemakerExperimentsLogger with other logger

In [ ]:
tensorboard_logger = TensorBoardLogger()
with Run(experiment_name="testExperiment", run_name="testRun2"):
    logger = SagemakerExperimentsLogger()
    trainer = Trainer(logger=[logger, tensorboard_logger])